In [235]:
from SeqEN2.session import Session
from SeqEN2.model.model import Model
from SeqEN2.autoencoder.autoencoder import Autoencoder
from glob import glob
from os import system
from torch import no_grad
from torch import argmax
from torch import sum as torch_sum
import torch
import numpy as np
import cProfile
import pstats

In [236]:
class NewAutoencoder(Autoencoder):
    def test_batch(self, input_vals, device, input_noise=0.0, wandb_log=True):
        """
        Test a single batch of data, this will move into autoencoder
        :param input_vals:
        :return:
        """
        with no_grad():
            input_ndx, one_hot_input = self.transform_input(
                input_vals, device, input_noise=input_noise
            )
            reconstructor_output = self.forward_test(one_hot_input)
            reconstructor_loss = self.criterion_NLLLoss(
                reconstructor_output, input_ndx.reshape((-1,))
            )
            # reconstructor acc
            reconstructor_ndx = argmax(reconstructor_output, dim=1)
            return reconstructor_ndx

            # reconstructor_accuracy = (
            #     torch_sum(reconstructor_ndx == input_ndx.reshape((-1,)))
            #     / reconstructor_ndx.shape[0]
            # )
            # # reconstruction_loss, discriminator_loss, classifier_loss
            # if wandb_log:
            #     wandb.log({"test_reconstructor_loss": reconstructor_loss.item()})
            #     wandb.log(
            #         {"test_reconstructor_accuracy": reconstructor_accuracy.item()}
            #     )
            # else:
            #     return reconstructor_loss, reconstructor_accuracy
            # # clean up
            # del reconstructor_loss
            # del reconstructor_output
            return


class NewModel(Model):
    def build_model(self, model_type, arch):
        if model_type == "AE":
            self.autoencoder = NewAutoencoder(self.d0, self.d1, self.dn, self.w, arch)
        # elif model_type == "AAE":
        #     self.autoencoder = AdversarialAutoencoder(
        #         self.d0, self.d1, self.dn, self.w, arch
        #     )
        # elif model_type == "AAEC":
        #     self.autoencoder = AdversarialAutoencoderClassifier(
        #         self.d0, self.d1, self.dn, self.w, arch
        #     )
        self.autoencoder.to(self.device)

    def test(self, num_test_items=1, input_noise=0.0):
        for test_batch in self.data_loader.get_test_batch(
            num_test_items=num_test_items
        ):
            results = self.autoencoder.test_batch(
                test_batch, self.device, input_noise=input_noise, wandb_log=False
            )
            return results


class NewSession(Session):
    def add_model(self, name, arch, model_type, d0=21, d1=8, dn=10, w=20):
        arch = self.load_arch(arch)
        if self.model is None:
            self.model = NewModel(name, arch, model_type, d0=d0, d1=d1, dn=dn, w=w)

    def load_data(self, dataset_name):
        data_files = sorted(glob(str(Model.root) + f"/data/{dataset_name}/*.csv.gz"))[
            :2
        ]
        self.model.load_data(dataset_name, data_files)

    def test(self, num_test_items=1, input_noise=0.0):
        return self.model.test(num_test_items=num_test_items, input_noise=input_noise)

In [237]:
session = NewSession()
model_name = "dummy"
arch = "gen1"
model_type = "AE"
dataset = "w_20_KeggSeq_ndx_wpACT"
run_title = "prototyping_consensus_acc"
w = 20

In [238]:
session.add_model(model_name, arch, model_type)
session.load_data(dataset)
result = session.test(num_test_items=1, input_noise=0.0)

In [239]:
result

tensor([13, 13,  5,  ..., 13, 13, 13])

In [240]:
result.reshape((-1, w))

tensor([[13, 13,  5,  ..., 13, 13, 13],
        [13, 13,  5,  ..., 13, 13, 13],
        [13, 13,  5,  ..., 13, 13, 13],
        ...,
        [13, 13,  5,  ..., 13, 13, 13],
        [13, 13,  5,  ..., 13, 13, 13],
        [13, 13,  5,  ..., 13, 13, 13]])

In [241]:
w = 5
l = 20
ol = l - w + 1
output = torch.randint(0, w - 1, (ol, w)).long()

print(output)


def consensus(output, ndx, w):
    f_size = min(l - ndx, ndx + 1)
    if f_size > w:
        f_size = w
    f = torch.fliplr(torch.eye(f_size).long())
    # print(f_size)
    r_min = max(0, ndx - w + 1)
    r_max = r_min + f_size
    r_indices = torch.tensor(range(r_min, r_max))
    c_min = max(0, ndx - ol + 1)
    c_max = min(ndx, w - 1) + 1
    c_indices = torch.tensor(range(c_min, c_max))
    sub_result = torch.index_select(
        torch.index_select(output, 0, r_indices), 1, c_indices
    )
    print(torch.mode(torch.diagonal(torch.fliplr(f * sub_result))).values.item())
    print("****************")

tensor([[3, 0, 3, 0, 1],
        [2, 0, 1, 3, 1],
        [0, 2, 3, 0, 2],
        [1, 1, 3, 3, 0],
        [3, 3, 3, 1, 1],
        [3, 1, 0, 0, 1],
        [0, 1, 2, 2, 1],
        [2, 0, 3, 1, 1],
        [0, 1, 0, 2, 0],
        [0, 3, 1, 0, 1],
        [3, 3, 1, 0, 3],
        [1, 0, 2, 2, 1],
        [0, 2, 1, 3, 2],
        [2, 2, 3, 0, 3],
        [0, 2, 3, 2, 2],
        [0, 3, 1, 3, 1]])


In [257]:
def consensus(output, ndx, w):
    ol = len(output)
    l = ol + w - 1
    f_size = min(l - ndx, ndx + 1)
    if f_size > w:
        f_size = w
    f = torch.fliplr(torch.eye(f_size).long())
    r_min = max(0, ndx - w + 1)
    r_max = r_min + f_size
    r_indices = torch.tensor(range(r_min, r_max))
    c_min = max(0, ndx - ol + 1)
    c_max = min(ndx, w - 1) + 1
    c_indices = torch.tensor(range(c_min, c_max))
    sub_result = torch.index_select(
        torch.index_select(output, 0, r_indices), 1, c_indices
    )
    return torch.mode(torch.diagonal(torch.fliplr(f * sub_result))).values


def acc(seq, output, w):
    n = 0
    for i, ndx in enumerate(seq):
        consensus_ndx = consensus(output, i, w)

        if ndx == consensus_ndx:
            n += 1
        # print(i, ndx, consensus_ndx, n)
    return n / len(seq)


def main(w, l):
    ol = l - w + 1
    output = torch.randint(0, 21, (ol, w)).long()
    seq = torch.randint(0, 21, (l,)).long()
    consensus_acc = acc(seq, output, w)
    # print(consensus_acc)


main(20, 100)

In [262]:
l = 100
w = 20
r = 100
job = f"tensor_{w}_{l}_{r}"
job = f"np_{w}_{l}_{r}"

with cProfile.Profile() as pr:
    for _ in range(r):
        main(w, l)
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.print_stats()
filename = f"./profiling_{job}.prof"
stats.dump_stats(filename)

         201602 function calls in 1.079 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    20000    0.192    0.000    0.192    0.000 {built-in method tensor}
    10000    0.180    0.000    0.953    0.000 /var/folders/kp/v03j_nl16632jlw_xj5z___00000gn/T/ipykernel_24590/472421105.py:1(consensus)
    20000    0.128    0.000    0.128    0.000 {built-in method fliplr}
    10000    0.125    0.000    0.125    0.000 {built-in method mode}
      100    0.111    0.001    1.074    0.011 /var/folders/kp/v03j_nl16632jlw_xj5z___00000gn/T/ipykernel_24590/472421105.py:17(acc)
    20000    0.107    0.000    0.107    0.000 {built-in method index_select}
    10200    0.066    0.000    0.066    0.000 {method 'long' of 'torch._C._TensorBase' objects}
    10000    0.044    0.000    0.044    0.000 {built-in method eye}
    10000    0.034    0.000    0.034    0.000 {built-in method diagonal}
    10100    0.029    0.000    0.047    0.000 /Users/ha

In [263]:
system(f"snakeviz {filename}")

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fhadinayebi%2FR%2FEG_HN_R%2FLabNoteBook%2F20211025%3AA%2FSeqEN%2Fnotebooks%2Fprofiling_np_20_100_100.prof

Bye!


0

In [10]:
run_dir = session.root / "models" / f"{model_name}" / "versions" / f"{run_title}"

In [12]:
system(f"rm -r {str(run_dir)}")

0